In [3]:
import torch
from torch import nn
from model.pointpillars import PointPillars
import open3d as o3d
import numpy as np
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
class PillarFeatureExtractor(nn.Module):
    def __init__(self, original_model: PointPillars, fc_dim=512):
        super().__init__()
        self.pillar_layer = original_model.pillar_layer
        self.pillar_encoder = original_model.pillar_encoder
        self.backbone = original_model.backbone
        self.neck = original_model.neck
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(384, fc_dim)

    def forward(self, batched_pts):
        timings = {}

        start_time = time.time()
        pillars, coors_batch, npoints_per_pillar = self.pillar_layer(batched_pts)
        timings["pillar_layer"] = time.time() - start_time

        start_time = time.time()
        pillar_features = self.pillar_encoder(pillars, coors_batch, npoints_per_pillar)
        timings["pillar_encoder"] = time.time() - start_time

        start_time = time.time()
        xs = self.backbone(pillar_features)
        timings["backbone"] = time.time() - start_time

        start_time = time.time()
        feats = self.neck(xs)
        timings["neck"] = time.time() - start_time

        start_time = time.time()
        pooled = self.pool(feats)
        pooled = pooled.view(pooled.size(0), -1)
        timings["pooling"] = time.time() - start_time

        start_time = time.time()
        out_512 = self.fc(pooled)
        timings["fc"] = time.time() - start_time

        # Print the timing information
        for stage, duration in timings.items():
            print(f"{stage}: {duration:.6f} seconds")

        return out_512



In [27]:
# Assuming `model` is the PointPillars model
pretrained_model_path = r"C:\Users\hussa\OneDrive\Desktop\Projects\ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data\poinpillar\pre_trained\epoch_160.pth"
model_state_dict = torch.load(pretrained_model_path, map_location=torch.device('cpu'))

# Initialize the PointPillars model
model = PointPillars(
            nclasses=3,
            voxel_size=[0.16, 0.16, 4],
            point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
            max_num_points=32,
            max_voxels=(16000, 40000)
        )

model_state_dict = torch.load(pretrained_model_path, map_location=torch.device('cuda'), weights_only=True)

# Load the filtered state dict into the model (without the head weights)
model.load_state_dict(model_state_dict, strict=False)


    

FCModel(
  (pointpillars): PointPillars(
    (pillar_layer): PillarLayer(
      (voxel_layer): Voxelization(
        (voxelizer): HardVoxelization()
      )
    )
    (pillar_encoder): PillarEncoder(
      (conv): Conv1d(9, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
    (backbone): Backbone(
      (multi_blocks): ModuleList(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
          (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      

In [31]:
# Function to load and preprocess PCD file with grayscale intensity
def load_pcd_file(pcd_path):
    # Load the point cloud data using Open3D
    pcd = o3d.io.read_point_cloud(pcd_path)

    # Convert points to numpy array
    points = np.asarray(pcd.points)  # (N, 3)

    # Check if grayscale intensity is available (as colors)
    if pcd.has_colors():
        # Use one channel (grayscale) from the RGB values
        colors = np.asarray(pcd.colors)  # (N, 3)
        grayscale_intensity = np.mean(colors, axis=1, keepdims=True)  # Convert RGB to grayscale
        data = np.hstack((points, grayscale_intensity))  # Combine points and grayscale intensity
    else:
        raise ValueError("The point cloud does not contain grayscale intensity information.")

    return data


def preprocess_point_cloud(point_cloud):
    """
    Reads .pcd with x, y, z + intensity embedded in colors.
    Returns a PyTorch tensor of shape [1, N, 4], where N is the number of points.
    """
    
    # Directly create the tensor for xyz and intensity
    xyz_tensor = torch.tensor(point_cloud[:, :3], dtype=torch.float32).to('cuda')  # Shape: (N, 3)
    intensity_tensor = torch.tensor(point_cloud[:, 3], dtype=torch.float32).reshape(-1, 1).to('cuda')  # Shape: (N, 1)

    # Remove points with NaN values in xyz or intensity
    valid_mask = ~torch.isnan(xyz_tensor).any(dim=1) & ~torch.isnan(intensity_tensor).any(dim=1)
    xyz_tensor = xyz_tensor[valid_mask]
    intensity_tensor = intensity_tensor[valid_mask]

    # Print min and max values for x, y, z
    min_x = torch.min(xyz_tensor[:, 0])  # Minimum value in x
    min_y = torch.min(xyz_tensor[:, 1])  # Minimum value in y
    min_z = torch.min(xyz_tensor[:, 2])  # Minimum value in z
    max_x = torch.max(xyz_tensor[:, 0])  # Maximum value in x
    max_y = torch.max(xyz_tensor[:, 1])  # Maximum value in y
    max_z = torch.max(xyz_tensor[:, 2])  # Maximum value in z

    print(f"Min values - x: {min_x.item()}, y: {min_y.item()}, z: {min_z.item()}")
    print(f"Max values - x: {max_x.item()}, y: {max_y.item()}, z: {max_z.item()}")

    # Concatenate xyz and intensity tensors directly
    points_tensor = torch.cat([xyz_tensor, intensity_tensor], dim=1)  # Shape: (N, 4)

    # Add batch dimension to make it [1, N, 4]
    points_tensor = points_tensor.unsqueeze(0)  # Shape: (1, N, 4)

    return points_tensor

In [33]:
# Inference function
def run_inference(input_tensor):
    # Run the model inference
        feature_extractor = PillarFeatureExtractor(model).to("cuda")
        with torch.no_grad():
            features = feature_extractor(input_tensor)
        output = features[0]  

        return output

In [34]:
# Example usage
pcd_path = r"C:\Users\hussa\Downloads\rosbag2_2024_10_22-18_21_24_0_pcd_1729614085.243545856.pcd"
points = load_pcd_file(pcd_path)

x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])

print(x_min, x_max, y_min, y_max)
x_range = x_max - x_min
y_range = y_max - y_min

print(x_range, y_range)

# Preprocess the point cloud data
input_tensor = preprocess_point_cloud(points)

# Run inference
features = run_inference(input_tensor)
 
print(features.shape)
print(f'fea {features}')


-31.77777671813965 54.69438552856445 -92.22269439697266 76.7667465209961
86.4721622467041 168.98944091796875
Input tensor shape: torch.Size([1, 13191, 3])
torch.Size([512])
fea tensor([-0.0574,  0.1601, -0.0901,  0.1384, -0.1569,  0.0990, -0.1710, -0.1188,
        -0.0103, -0.1028, -0.0946,  0.0488,  0.2588,  0.1035, -0.0120,  0.0356,
        -0.1061, -0.0077,  0.0708, -0.0990,  0.1504, -0.1407,  0.1077, -0.1237,
         0.0226,  0.0868, -0.1475,  0.1259, -0.0586,  0.1449, -0.1752,  0.1087,
        -0.1315, -0.1191,  0.0941,  0.1225,  0.0471, -0.2363, -0.2305,  0.0786,
         0.1289, -0.0737,  0.0893, -0.0244,  0.1045,  0.0943, -0.0518,  0.0087,
         0.2356,  0.0033,  0.2226, -0.0128, -0.0178,  0.0494, -0.1500, -0.0361,
         0.1546, -0.0222, -0.0470, -0.1880,  0.1236, -0.0987,  0.0984, -0.1861,
        -0.2538,  0.1047, -0.1203,  0.1468, -0.0459,  0.0453,  0.2171,  0.1294,
        -0.0155, -0.0333, -0.0396, -0.0988, -0.0179,  0.0145, -0.2498,  0.2064,
         0.1072,  0.159